In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import keras
from keras.applications import imagenet_utils
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from IPython.display import Image

import pathlib
from pathlib import Path

In [ ]:
mobile = keras.applications.mobilenet.MobileNet()

In [ ]:
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size = (224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis = 0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [ ]:
def calculate_prediction(file_image):
    img = mpimg.imread(file_image)
    preprocessed_image = prepare_image(file_image)
    predictions = mobile.predict(preprocessed_image)
    results = imagenet_utils.decode_predictions(predictions)
    return results, img

In [ ]:
borzoi = 'data/archive/images/15-n02090622-borzoi/train/n02090622_10281.jpg'
chihuahua = 'data/archive/images/50-n02085620-Chihuahua/train/n02085620_10131.jpg'
german_shepherd = 'data/archive/images/100-n02106662-German_shepherd/train/n02106662_10338.jpg'
dog_list = [borzoi, chihuahua, german_shepherd]

In [ ]:
for elem in dog_list:
    print(elem)
    res, img = calculate_prediction(elem)
    imgplot = plt.imshow(img)
    plt.show()
    # print(res)
    for row in res:
        for col in row:
            print(col)


### Setup neural network

In [ ]:
base_model = MobileNet(weights = 'imagenet', include_top = False) # imports the mobilenet model and discards the last 1000 neuron layer.
x = base_model.output
x = GlobalAveragePooling2D()(x)
# add dense layers so that the model can learn more complex functions and classify for better results
x = Dense(1024, activation = 'relu')(x)
# dense layer 2
x = Dense(1024, activation = 'relu')(x)
# dense layer 3
x = Dense(512, activation = 'relu')(x)
# final layer with softmax activation
preds = Dense(2, activation = 'softmax')(x)

### 15 img for train

In [ ]:
model_15 = Model(inputs = base_model.input, outputs = preds)

for layer in model_15.layers:
    layer.trainable = False
for layer in model_15.layers[:20]:
    layer.trainable = False
for layer in model_15.layers[20:]:
    layer.trainable = True

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

train_generator = train_datagen.flow_from_directory(
    'data/archive/images/15-n02090622-borzoi/', 
    target_size=(224,224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

In [ ]:
model_15.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

step_size_train = train_generator.n//train_generator.batch_size
model_15.fit_generator(generator = train_generator, steps_per_epoch = step_size_train, epochs = 8, verbose = 2)

### 50 img for train

In [ ]:
model_50 = Model(inputs = base_model.input, outputs = preds)

for layer in model_50.layers:
    layer.trainable = False
for layer in model_50.layers[:20]:
    layer.trainable = False
for layer in model_50.layers[20:]:
    layer.trainable = True

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input) # included in our dependencies

train_generator = train_datagen.flow_from_directory(
    'data/archive/images/50-n02085620-Chihuahua/', 
    target_size=(224,224), 
    color_mode='rgb', 
    batch_size=32, 
    class_mode='categorical', 
    shuffle=True)

In [ ]:
model_50.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

step_size_train = train_generator.n//train_generator.batch_size
model_50.fit_generator(generator = train_generator, steps_per_epoch = step_size_train, epochs = 8, verbose = 2)

### 100 img for train

In [ ]:
model_100 = Model(inputs = base_model.input, outputs = preds)

for layer in model_100.layers:
    layer.trainable = False
for layer in model_100.layers[:20]:
    layer.trainable = False
for layer in model_100.layers[20:]:
    layer.trainable = True

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input) # included in our dependencies

train_generator = train_datagen.flow_from_directory(
    'data/archive/images/100-n02106662-German_shepherd/', 
    target_size = (224,224), 
    color_mode = 'rgb', 
    batch_size = 32, 
    class_mode = 'categorical', 
    shuffle = True)

In [ ]:
model_100.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

step_size_train = train_generator.n//train_generator.batch_size
model_100.fit_generator(generator = train_generator, steps_per_epoch = step_size_train, epochs = 8, verbose = 2)

## Test

In [ ]:
# test 15
m = {}
borzoi_test = "data/archive/images/15-n02090622-borzoi/test/"
for path in pathlib.Path(borzoi_test).iterdir():
    if path.is_file():
        # print(path)
        new_image = load_image(path)
        pred = model_15.predict(new_image)
        # print(pred)
        m[path] = pred[0]

runtime_error = []
value_error = []
for key, value in m.items():
    runtime_error.append(value[0])
    value_error.append(value[1])
data_15 = {
    'Image': list(m.keys()),
    'Runtime Error': list(runtime_error),
    'Value Error': list(value_error)
}
df_15 = pd.DataFrame.from_dict(data_15)
dfAIC = df_15.sort_values(by=['Runtime Error'])

In [ ]:
# test 50
m = {}
chihuahua_test = "data/archive/images/50-n02085620-Chihuahua/test"
for path in pathlib.Path(chihuahua_test).iterdir():
    if path.is_file():
        # print(path)
        new_image = load_image(path)
        pred = model_50.predict(new_image)
        # print(pred)
        m[path] = pred[0]

runtime_error = []
value_error = []
for key, value in m.items():
    runtime_error.append(value[0])
    value_error.append(value[1])
data_50 = {
    'Image': list(m.keys()),
    'Runtime Error': list(runtime_error),
    'Value Error': list(value_error)
}
df_50 = pd.DataFrame.from_dict(data_50)
dfAIC = df_50.sort_values(by=['Runtime Error'])

In [ ]:
# test 100
m = {}
shepherd_test = "data/archive/images/100-n02106662-German_shepherd/test/"
for path in pathlib.Path(shepherd_test).iterdir():
    if path.is_file():
        # print(path)
        new_image = load_image(path)
        pred = model_100.predict(new_image)
        # print(pred)
        m[path] = pred[0]

runtime_error = []
value_error = []
for key, value in m.items():
    runtime_error.append(value[0])
    value_error.append(value[1])
data_100 = {
    'Image': list(m.keys()),
    'Runtime Error': list(runtime_error),
    'Value Error': list(value_error)
}
df_100 = pd.DataFrame.from_dict(data_100)
dfAIC = df_100.sort_values(by=['Runtime Error'])

In [ ]:
df_15.head(10)

In [ ]:
df_50.head(10)

In [ ]:
df_100.head(10)

### function to load image

In [ ]:
def load_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]
    return img_tensor